# Introdução

# Dependências

In [43]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install tweepy
!{sys.executable} -m pip install requests

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [147]:
import pandas as pd
import pandas.api.types as ptypes
import numpy as np
import requests
import tweepy

import json
import os

# Coleta

## Previsões

In [174]:
predictions_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
filename = 'image_predictions.tsv'
if not os.path.exists(filename):
    response = requests.get(predictions_url)
    with open(filename, 'wb') as file:
        file.write(response.content)

In [175]:
df_predictions = pd.read_csv('image_predictions.tsv', sep='\t')
df_predictions_clean = df_predictions.copy()

## Tweets

In [176]:
df_tweets = pd.read_csv('twitter-archive-enhanced.csv')
df_tweets_clean = df_tweets.copy()

## Contagem de retweets e favoritos

In [177]:
consumer_key = os.environ['TWITTER_CONSUMER_KEY']
consumer_secret = os.environ['TWITTER_CONSUMER_SECRET']
access_token = os.environ['TWITTER_ACCESS_TOKEN']
access_token_secret = os.environ['TWITTER_ACCESS_TOKEN_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

twitter_api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [178]:
# Função para mapear o resultado de cada chamada API para um resultado JSON
def get_retweet_counts(tweet_id):
    try:
        tweet_status = twitter_api.get_status(tweet_id, tweet_mode='extended')
        return tweet_status._json
    except tweepy.TweepError:
        pass

In [179]:
filename = 'tweet_json.txt'

if not os.path.exists(filename):
    tweet_counts = df_tweets_clean['tweet_id'].apply(lambda tweet_id: get_retweet_counts(tweet_id))
    with open(filename, 'w+') as handle:
        for tweet_count in tweet_counts:
            handle.write(json.dumps(tweet_count) + '\n')

tweet_data = []
with open(filename, 'r') as handle:
    for line in handle:
        tweet_count = json.loads(line)
        tweet_data.append(tweet_count)

In [180]:
filtered_tweet_data = list(filter(lambda r: r is not None, tweet_data))
df_tweet_counts = pd.DataFrame(filtered_tweet_data)
df_tweet_counts_clean = df_tweet_counts[['id', 'favorite_count', 'retweet_count']].copy()

# Fusão dos dados de tweet e contagem de retweet e favoritos

In [181]:
df_tweet_counts_clean = df_tweet_counts_clean.rename(columns={'id': 'tweet_id'})
df_tweets_clean = pd.merge(df_tweets_clean, df_tweet_counts_clean, how='left', on='tweet_id')

# Avaliação

## Estrutura do DataFrame de previsões

In [182]:
df_predictions_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


## Estrutura do DataFrame de tweets

In [183]:
df_tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
favorite_count                23

## Identificação de Retweets

In [158]:
df_retweets = df_tweets_clean[df_tweets_clean.in_reply_to_status_id.notnull()]
df_retweets.shape[0]

78

## Denominador com valor abaixo de 10

In [159]:
df_denominator_below_10 = df_tweets_clean[df_tweets_clean.rating_denominator < 10]
df_denominator_below_10.shape[0]

3

## Denominador com valor acima de 10

In [160]:
df_denominator_above_10 = df_tweets_clean[df_tweets_clean.rating_denominator > 10]
df_denominator_above_10.shape[0]

20

## Tweets sem referência do nome do animal

In [161]:
df_no_name = df_tweets_clean[df_tweets_clean.name == 'None']
df_no_name.shape[0]

745

# Documentação dos Problemas de Qualidade

## Problemas de Qualidade

1. 78 tweets são retweets (seção "Identificação de Retweets")
2. Tipo das colunas "in_reply_to_status_id", "in_reply_to_user_id", "retweeted_status_id", "retweeted_status_user_id" deve ser alterado de float64 para int64 (seção "Estrutura do DataFrame de tweets")
3. Tipo das colunas "favorite_count" e "retweet_count" deve ser alterado de float64 para int64, após fusão dos dados (seção "Estrutura do DataFrame de tweets")
4. Tipo das colunas "timestamp" e "retweeted_status_timestamp" deve ser alterado de string (object) para datetime
5. 3 tweets com denominador abaixo de 10 (seção "Denominador com valor abaixo de 10")
6. 20 tweets com denominador acima de 10 (seção "Denominador com valor acima de 10")
7. 745 tweets não contém o nome do animal a qual se referem (seção "Tweets sem referência do nome do animal")
8. Alguns tweets não possuem predições (**Não pode ser limpo**)

## Problemas de Arrumação

1. DataFrame com as previsões contem 9 colunas relacionadas aos valores de previsão, onde apenas 3 colunas devem ser utilizadas para normalizar estes valores (previsão, intervalo de confiança e valor booleano indicando se a previsão é uma raça de cachorro) (seção "Estrutura do DataFrame de previsões")
2. DataFrame com os tweets contem 4 colunas relacionadas ao "estágio" do animal, podendo ser normalizada para apenas uma coluna "estágio" (seção "Estrutura do DataFrame de tweets")

# Limpeza

## Problemas de Qualidade

### 1. Descartar retweets (não são o foco da análise)

In [187]:
df_tweets_clean = df_tweets_clean[df_tweets_clean.in_reply_to_status_id.isnull()]
df_tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                      2278 non-null int64
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     2278 non-null object
source                        2278 non-null object
text                          2278 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2274 non-null object
rating_numerator              2278 non-null int64
rating_denominator            2278 non-null int64
name                          2278 non-null object
doggo                         2278 non-null object
floofer                       2278 non-null object
pupper                        2278 non-null object
puppo                         2278 non-null object
favorite_count                2262

#### Teste

In [188]:
assert len(df_tweets_clean[df_tweets_clean.in_reply_to_status_id.notnull()]) == 0

### 2. Ajustar tipo de dados das colunas "in_reply_to_status_id", "in_reply_to_user_id", "retweeted_status_id", "retweeted_status_user_id" (float para int)

In [189]:
df_tweets_clean.in_reply_to_status_id = df_tweets_clean.in_reply_to_status_id.fillna(0).astype(int)
df_tweets_clean.in_reply_to_user_id = df_tweets_clean.in_reply_to_user_id.fillna(0).astype(int)
df_tweets_clean.retweeted_status_id = df_tweets_clean.retweeted_status_id.fillna(0).astype(int)
df_tweets_clean.retweeted_status_user_id = df_tweets_clean.retweeted_status_user_id.fillna(0).astype(int)
df_tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                      2278 non-null int64
in_reply_to_status_id         2278 non-null int64
in_reply_to_user_id           2278 non-null int64
timestamp                     2278 non-null object
source                        2278 non-null object
text                          2278 non-null object
retweeted_status_id           2278 non-null int64
retweeted_status_user_id      2278 non-null int64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2274 non-null object
rating_numerator              2278 non-null int64
rating_denominator            2278 non-null int64
name                          2278 non-null object
doggo                         2278 non-null object
floofer                       2278 non-null object
pupper                        2278 non-null object
puppo                         2278 non-null object
favorite_count                2262

#### Teste

In [190]:
assert len(df_tweets_clean[df_tweets_clean.in_reply_to_status_id.isnull()]) == 0
assert len(df_tweets_clean[df_tweets_clean.in_reply_to_user_id.isnull()]) == 0
assert len(df_tweets_clean[df_tweets_clean.retweeted_status_id.isnull()]) == 0
assert len(df_tweets_clean[df_tweets_clean.retweeted_status_user_id.isnull()]) == 0

assert ptypes.is_integer_dtype(df_tweets_clean.in_reply_to_status_id)
assert ptypes.is_integer_dtype(df_tweets_clean.in_reply_to_user_id)
assert ptypes.is_integer_dtype(df_tweets_clean.retweeted_status_id)
assert ptypes.is_integer_dtype(df_tweets_clean.retweeted_status_user_id)

### 3. Ajustar tipo de dados das colunas "favorite_count" e "retweet_count" (float para int)

In [191]:
df_tweets_clean.favorite_count = df_tweets_clean.favorite_count.fillna(0).astype(int)
df_tweets_clean.retweet_count = df_tweets_clean.retweet_count.fillna(0).astype(int)

#### Teste

In [192]:
assert len(df_tweets_clean[df_tweets_clean.favorite_count.isnull()]) == 0
assert len(df_tweets_clean[df_tweets_clean.retweet_count.isnull()]) == 0

assert ptypes.is_integer_dtype(df_tweets_clean.favorite_count)
assert ptypes.is_integer_dtype(df_tweets_clean.retweet_count)

### 4. Ajustar tipo de dados das colunas "timestamp" e "retweeted_status_timestamp" (string para datetime)

In [193]:
df_tweets_clean.timestamp = pd.to_datetime(df_tweets_clean.timestamp)
df_tweets_clean.retweeted_status_timestamp = pd.to_datetime(df_tweets_clean.retweeted_status_timestamp)
df_tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                      2278 non-null int64
in_reply_to_status_id         2278 non-null int64
in_reply_to_user_id           2278 non-null int64
timestamp                     2278 non-null datetime64[ns]
source                        2278 non-null object
text                          2278 non-null object
retweeted_status_id           2278 non-null int64
retweeted_status_user_id      2278 non-null int64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2274 non-null object
rating_numerator              2278 non-null int64
rating_denominator            2278 non-null int64
name                          2278 non-null object
doggo                         2278 non-null object
floofer                       2278 non-null object
pupper                        2278 non-null object
puppo                         2278 non-null object
favorite_count    

#### Teste

In [194]:
assert ptypes.is_datetime64_any_dtype(df_tweets_clean.timestamp)
assert ptypes.is_datetime64_any_dtype(df_tweets_clean.retweeted_status_timestamp)

### 5. Ajustar tweets com denominador abaixo de 10

#### Teste

### 6. Ajustar tweets com denominador acima de 10

In [195]:
def adjust_denominator_to_10(row):
    current_numerator = row['rating_numerator']
    current_denominator = row['rating_denominator']
    if current_denominator > 10:
        new_numerator = int(current_numerator * 10 / current_denominator)
        row['rating_numerator'] = new_numerator
        row['rating_denominator'] = 10
    return row

In [196]:
df_tweets_clean = df_tweets_clean.apply(adjust_denominator_to_10, axis=1)
df_tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                      2278 non-null int64
in_reply_to_status_id         2278 non-null int64
in_reply_to_user_id           2278 non-null int64
timestamp                     2278 non-null datetime64[ns]
source                        2278 non-null object
text                          2278 non-null object
retweeted_status_id           2278 non-null int64
retweeted_status_user_id      2278 non-null int64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2274 non-null object
rating_numerator              2278 non-null int64
rating_denominator            2278 non-null int64
name                          2278 non-null object
doggo                         2278 non-null object
floofer                       2278 non-null object
pupper                        2278 non-null object
puppo                         2278 non-null object
favorite_count    

#### Teste

In [197]:
assert len(df_tweets_clean[df_tweets_clean.rating_denominator > 10]) == 0

## Problemas de Arrumação

### 1. Ajuste das predições

In [198]:
df_predictions_1 = df_predictions_clean[["tweet_id", "jpg_url", "img_num", "p1", "p1_conf", "p1_dog"]].copy()
df_predictions_2 = df_predictions_clean[["tweet_id", "jpg_url", "img_num", "p2", "p2_conf", "p2_dog"]].copy()
df_predictions_3 = df_predictions_clean[["tweet_id", "jpg_url", "img_num", "p3", "p3_conf", "p3_dog"]].copy()

df_predictions_1.rename(columns={'p1': 'prediction', 'p1_conf': 'confidence_interval', 'p1_dog': 'is_dog_race'}, inplace=True)
df_predictions_2.rename(columns={'p2': 'prediction', 'p2_conf': 'confidence_interval', 'p2_dog': 'is_dog_race'}, inplace=True)
df_predictions_3.rename(columns={'p3': 'prediction', 'p3_conf': 'confidence_interval', 'p3_dog': 'is_dog_race'}, inplace=True)

df_predictions_clean = pd.concat([df_predictions_1, df_predictions_2, df_predictions_3])
df_predictions_clean.shape[0]

6225

#### Teste

In [199]:
df_no_name.sample(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,favorite_count,retweet_count
1972,673240798075449344,NaN,NaN,2015-12-05 20:41:29 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Magical floating dog here. Very calm. Always h...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/673240798...,6,10,None,None,None,None,None,1464.0,764.0
